<a href="https://colab.research.google.com/github/thiagoscerqueira/fiap_techchallenge_03/blob/main/fine_tuning_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset
dataset = load_dataset('thiagoscerqueira/product_reviews_tst', data_files={'train': 'reviews.csv'})



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


reviews.csv:   0%|          | 0.00/626 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['review_text', 'label'],
        num_rows: 10
    })
})


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Split dataset into train and validation sets (assuming 'dataset' is defined)
split_dataset = dataset['train'].train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

def preprocess_function(examples):
    return tokenizer(examples['review_text'], truncation=True, padding='max_length')

# Apply the preprocessing function to your datasets
train_dataset = train_dataset.map(preprocess_function, batch_size=16)
eval_dataset = eval_dataset.map(preprocess_function, batch_size=16)
print(train_dataset)
print(eval_dataset)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['review_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 8
})
Dataset({
    features: ['review_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2
})


In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    evaluation_strategy="epoch",     # Evaluation strategy (per epoch)
    per_device_train_batch_size=16,  # Training batch size
    per_device_eval_batch_size=16,   # Evaluation batch size
    num_train_epochs=3,              # Number of training epochs
    logging_dir='./logs',            # Logging directory
    logging_steps=10,
)

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,      # Training data
    eval_dataset=eval_dataset,        # Evaluation data
)




/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.590749
2,No log,0.544026
3,No log,0.520519


TrainOutput(global_step=3, training_loss=0.5255533854166666, metrics={'train_runtime': 12.6364, 'train_samples_per_second': 1.899, 'train_steps_per_second': 0.237, 'total_flos': 6314665328640.0, 'train_loss': 0.5255533854166666, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5205190777778625,
 'eval_runtime': 0.0999,
 'eval_samples_per_second': 20.012,
 'eval_steps_per_second': 10.006,
 'epoch': 3.0}

In [ ]:
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')


('./fine-tuned-model/tokenizer_config.json',
 './fine-tuned-model/special_tokens_map.json',
 './fine-tuned-model/vocab.txt',
 './fine-tuned-model/added_tokens.json')

In [ ]:
!pip install torch


In [ ]:
import torch
inputs = tokenizer("This product is awesome.", return_tensors="pt").to('cuda')
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)

# Print prediction
print(predictions)


tensor([0], device='cuda:0')
